<a href="https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/notebooks/00_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Using geemap with Google Colab

## Introduction

This notebook demonstrates how to use [geemap](https://geemap.org) with Google Colab. To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial).

## Install geemap

The geemap package is pre-installed in Google Colab and is updated to the latest minor or major release every few weeks. If you would like to install the latest version, you can uncomment and run the following command:

In [ ]:
# %pip install -U geemap

In [21]:
pip install geedim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.3 MB/s eta 0:00:00


## Import libraries

Import the Earth Engine Python API.

In [2]:
import ee

## EE Authentication

Running the following cell will start the Earth Engine authentication. Follow the instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication) to authenticate Earth Engine.

In [8]:
ee.Authenticate()

True

In [10]:
ee.Initialize(project='advance-river-408203')

## Core features

You can import the geemap package using the following conventions:

- To import only the core features of geemap, use: `import geemap.core as geemap`
- To import all the features of geemap, use: `import geemap`

Let's import the geemap package using the second option:

In [18]:
import geemap

## Basemap selector

Select a basemap from the dropdown list and add it to the map.

In [12]:
m = geemap.Map()
m.add("basemap_selector")
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [15]:
table = ee.FeatureCollection('users/pxu7709/huanghe')  # 请替换为您的 FeatureCollection 路径
Albufera1 = table.geometry()

# 定义调色板
palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301'
]

# 定义辅助函数
def getQABits(image, start, end, newName):
    """从 QA 波段中提取指定位"""
    pattern = 0
    for i in range(start, end + 1):
        pattern += 2 ** i
    return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

def maskEmptyPixels(image):
    """掩膜掉没有观测值的像素"""
    withObs = image.select('num_observations_1km').gt(0)
    return image.updateMask(withObs)

def maskClouds(image):
    """掩膜掉云像素，保留清晰像素"""
    QA = image.select('state_1km')
    internalCloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    return image.updateMask(internalCloud.eq(0))

def masknoLand(image):
    """掩膜掉非陆地像素，保留陆地像素"""
    QA = image.select('state_1km')
    landWaterFlag = getQABits(QA, 3, 5, 'Land/water flag')
    nonLand = landWaterFlag.expression("b(0) > 1 || b(0) == 0")
    return image.updateMask(nonLand.eq(0))

def maskmod15(image1):
    """掩膜掉 LAI 产品中的无效像素"""
    qc = image1.select('FparLai_QC')
    maskbad = qc.eq(0).Or(qc.eq(2)).Or(qc.eq(32)).Or(qc.eq(34))
    return image1.updateMask(maskbad)

def clipImage(image):
    """将图像裁剪到研究区域"""
    return image.clip(Albufera1)


def addKNDVI(image):
    """计算 kNDVI"""
    red = image.select('sur_refl_b01')
    nir = image.select('sur_refl_b02')
    D2 = nir.subtract(red).pow(2).rename('d2')
    sigma = 0.15
    kndvi = D2.divide(sigma).divide(sigma).divide(4.0).tanh().rename('kndvi')
    return image.addBands(kndvi)

# 加载 MODIS 数据并应用预处理
collection = ee.ImageCollection("MODIS/006/MOD09GA") \
    .filterDate('2018-01-01', '2019-01-11') \
    .map(clipImage) \
    .map(maskEmptyPixels) \
    .map(masknoLand) \
    .map(maskClouds)

# 加载并处理 LAI 数据
MODIS_LAI = ee.ImageCollection("MODIS/006/MCD15A3H") \
    .filterDate('2018-01-01', '2019-01-11') \
    .map(maskmod15) \
    .select('Lai')

# 缩放反射率
collectionCloudMasked = collection.select(
    ['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04']
).map(lambda image: image.multiply(ee.Image.constant(0.0001)) \
    .set('system:time_start', image.get('system:time_start')))

# 计算植被指数
collectionNDVI = collectionCloudMasked.map(addKNDVI)

# 计算平均值
mean_kndvi = collectionNDVI.select('kndvi').mean()

# 创建并配置地图
Map = geemap.Map()
Map.centerObject(Albufera1, 9)
Map.addLayer(mean_kndvi, {'min': 0, 'max': 1, 'palette': palette}, 'kNDVI')

# 显示地图
Map

Map(center=[39.432345322631356, 106.84057138282243], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [24]:
import os
work_dir = os.path.join(os.path.expanduser("E:\大三项目\data"), 'tif')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_tif = os.path.join(work_dir, "mean_kndvi.tif")
geemap.download_ee_image(
    image = mean_kndvi,
    filename = out_tif,
    region = Albufera1,
    crs="EPSG:4326",
    scale=1000,
)



mean_kndvi.tif: |          | 0.00/13.0M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [26]:
out_tif



'E:\\大三项目\\data/tif/mean_kndvi.tif'

## Layer Manager

Toggle the checkbox to show or hide the layer. Drag and move the slider to change the transparency level of the layer.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image("USGS/SRTMGL1_003")
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.add_layer(dem, vis_params, "SRTM DEM")
m.add_layer(states, {}, "US States")
m.add("layer_manager")
m

## Inspector

Click on the map to query Earth Engine data at a specific location.

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image("USGS/SRTMGL1_003")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.add_layer(dem, vis_params, "SRTM DEM")
m.add_layer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2.0},
    "Landsat 7",
)
m.add_layer(states, {}, "US States")
m.add("inspector")
m

## Layer Editor

You can change the visualization parameters of the Earth Engine data using the layer editor.

### Single-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.add_layer(dem, vis_params, "SRTM DEM")
m.add("layer_editor", layer_dict=m.ee_layers["SRTM DEM"])
m

### Multi-band image

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
m.add_layer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2.0},
    "Landsat 7",
)
m.add("layer_editor", layer_dict=m.ee_layers["Landsat 7"])
m

### Feature collection

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
m.add_layer(states, {}, "US States")
m.add("layer_editor", layer_dict=m.ee_layers["US States"])
m

## Draw control

You can draw shapes on the map using the draw control. The drawn features will be automatically converted to Earth Engine objects, which can be accessed in one of the following ways:

- To return the last drawn feature as an `ee.Geometry()`, use: `m._draw_control.last_geometry`
- To return all the drawn feature as an `ee.Feature()`, use: `m._draw_control.last_feature`
- To return all the drawn features as an `ee.FeatureCollection()`, use: `m._draw_control.collection`

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image("USGS/SRTMGL1_003")
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": "terrain",
}
m.add_layer(dem, vis_params, "SRTM DEM")
m.add("layer_manager")
m

Use the draw control to draw a polygon on the map.

In [ ]:
geometry = m._draw_control.last_geometry
geometry

In [ ]:
feature = m._draw_control.last_feature
feature

In [ ]:
collection = m._draw_control.collection
collection

Clip the DEM data using the drawn polygon.

In [ ]:
if geometry is not None:
    image = dem.clipToCollection(collection)
    m.layers[1].visible = False
    m.add_layer(image, vis_params, "Clipped DEM")
m